# ChiPy Mentorship 2 of 3

#### Learning and Project Clarity
Since Post 1, my time investment in this project has varied between learning data science at a high level and expanding my Python foundation through tutorials and project work. I have been working my way through the book, Data Science for Business, which nicely balances emphasis between high level discussion and gritty nuance. Importantly, the text provides language for speaking to my interests and experiences in analytics and defining my project stages and scope.

The analysis I’m conducting involves profiling and causal modeling. I want to know which data points correlate most strongly to student outcomes and AmeriCorps Member performance (see my previous post for information on my role at an educational non-profit). I am hoping I can then build a profile of successful AmeriCorps Members (ACMs) and use this model to recommend metrics and goals that better align to student outcomes and ACM performance. Additionaly, this analysis could help advocate to school partners for the school settings in which our AmeriCorps Members best serve students’ individualistic needs.

On the technical Python side, I have gained further experience with tuples, dictionaries, sets, and `with` control flows while adding further practice to the other basic types and `pandas` techniques I already knew well. The tutorials I followed include a Exploratory Data Analysis https://github.com/cmawer/pycon-2017-eda-tutorial from PyCon, the seaborn data visualization tutorial https://seaborn.pydata.org/tutorial.html, and a little natural langauge processing http://textblob.readthedocs.io/en/dev/.

Although these tutorials filled me with a sense of potential and accompishment, the feeling quickly faded in the absence of application. Of course, this is where the 'hard learning' occurs and meaningful progress is made. Following are two examples of the data I have processed in my project to date.

#### Getting Data
First I needed to get my data into Python. The majority of my data are stored on Salesforce or Excel workbooks through SharePoint.

Going into this project, I had already developed Python systems for getting and writing Excel documents to SharePoint. This was accomplished through a mapped network drive to the SharePoint server, allowing me to simply navigate the file structure as a local drive.

The Salesforce component was a bit more tricky, but with a little Python elbow grease, the `simple-salesforce` package was up to the task. This package allows me to query, create, and delete records, which has had a profound effect on my work pracitices. In a typical use of simple-salesforce, the user passes a traditional SOQL query into the Salesforce instance (in this case `cysh`) and performs the `query_all` function.

Here is the set-up:

In [4]:
import pandas as pd
from simple_salesforce import Salesforce

with open('C:\\Users\\City_Year\\Desktop\\salesforce_credentials.txt', 'r') as f:
    read_data = f.read()
    sf_creds = eval(read_data)

# our Salesforce instance is referred to as cyschoolhouse (cysh)
cysh = Salesforce(instance_url=sf_creds['instance_url'],
                  password=sf_creds['password'],
                  username=sf_creds['username'],
                  security_token=sf_creds['security_token'])

A simple query, for example:

In [ ]:
querystring = (f"SELECT Id, Name FROM Assesment__c") # Note the typo in Assesment__c, which cost me a good amount of time troubleshooting
query_return = cysh.query_all(querystring)

In [ ]:
type(query_return)

In [ ]:
query_return.keys()

In [ ]:
len(query_return['records'])

In my case, I'm working with datasets that are not very large. In the Chicago region, we serve fewer than 3000 students and enlist fewer than 300 AmeriCorps Members. Therefore I can query the entire Salesforce object (i.e. `Assesment__c`) with whatever fields I'm interested in and perform all the shaping and filtering in Python.

I added this convenient function to iterate over the query response and shape it as a dataframe:

In [5]:
def get_cysh_df(sf_object, sf_fields, rename_id=False, rename_name=False, sf=cysh):
    sf_fields_str = ", ".join(sf_fields)
    querystring = (f"SELECT {sf_fields_str} FROM {sf_object}")
    query_return = cysh.query_all(querystring)

    query_list = []
    for row in query_return['records']:
        record = []
        for column in sf_fields:
            col_data = row[column]
            record.append(col_data)
        query_list.append(record)
    
    df = pd.DataFrame(query_list, columns=sf_fields)
    
    if rename_id==True:
        df.rename(columns={'Id':sf_object}, inplace=True)
    if rename_name==True:
        df.rename(columns={'Name':(sf_object+'_Name')}, inplace=True)

    return df

The use of this function is as follows:

In [6]:
assessment_df = get_cysh_df('Assesment__c', ['Id', 'Name'])
assessment_df.head()

,Id,Name
0,a041a00000E9Y3xAAF,a041a00000E9Y3x
1,a041a00000EmNs0AAF,a041a00000EmNs0
2,a041a00000EmOf8AAF,a041a00000EmOf8
3,a041a00000FV7AaAAL,a041a00000FV7Aa
4,a041a00000FVLdgAAH,a041a00000FVLdg


In terms of assessment data, the next step will be determing student assessment progress by comparing prior year or start of year assessments to the recent winter assessment. Typically we set assessment goals fall to spring, so in this analysis I will need to calculate mid-year goals. Ideally, this analysis will also consider data from past years, but the feasbility of that is yet to be determined. To view my current progress on assessment data, view the IPython notebook in the repository for this project.

#### Cleaning Data
In my organization we employ four coaches who observe, provide feedback, and enter data for each AmeriCorps member at least once per month. If this data were cleaned and rolled together, I could determine whether the AmeriCorps Members who exercise best practices in tutoring are in fact more impactful with students (among many other interesting questions). The reason I put off this analysis is because the 26 Excel workbooks created to track coaching data were not designed with aggregation in mind. The records are indexed by first names and nicknames, headers are inconsistent (and multi-indexed), and the file structure does not reflect standardized organization.

The first task was to identify the file paths to each Excel workbook. This was accomplished using `os.walk()` to list all contents of the relevant SharePoint directories. Since there are only 26 files, and the filenames are not patterned, I chose to simply copy-paste the file paths I wanted into a list.

The next challenge was that these Excel workbooks do not contain consistent sheets. All workbooks contain months as sheet names, but some coaches split their workbook into two semesters. Instead of pandas' `pd.read_excel()`, I used `pd.ExcelFile()`. This allowed me to load the workbook once, then only pull the sheets whose names also occur in the set of sheets I'm interested in:

In [ ]:
for filepath in coach_xlsx_paths:
    with pd.ExcelFile(filepath) as xl:
        for sheet in list(set(xl.sheet_names) & set(sheet_months)):
            # Returns ordered dict
            heatmap = pd.read_excel(xl, header=[2,3], 
                                    sheet_name=sheet, 
                                    usecols=14, index_col=None)

To this process, I also added columns for the month (the sheet name) and the school (the directory in which the file resides), but again the school folder names were not standardized.

Enter `fuzzywuzzy`, a very hand package for finding and scoring text matches. I performed fuzzy matches between the folder name columns and the official school names from Salesforce like so: 

In [ ]:
# Load schools from salesforce
account_df = get_cysh_df('Account', ['Id', 'Name'])
account_df.rename(columns={'Id':'Organization__c', 'Name':'School'}, inplace=True)

d = []
for folder_name in set(heatmaps_df['Folder']):
    match = process.extract(folder_name, set(account_df['School']), scorer=fuzz.token_set_ratio, limit=1)
    match = [folder_name] + [x for tup in match for x in tup]
    d.append(match)
df = pd.DataFrame(d, columns=['Folder', 'School', 'Match_Score'])
df.head()

Using the same process, I was able to clean ACM names too. However, in this case there were several ambiguous matches, which I resolved by manually comparing names between the Excel workbook and the validated set of ACM names from Salesforce. I then fixed the mis-matched names in Excel by hand. You can also view this full process in the project repository.

#### Next Steps
Next in my analysis will be cleaning and combining of 2-3 more data sources. Then I will conduct exploratory data analysis, cyclically chasing my branching curiosities as they come. For example, I will experiment with various definitions of ACM success to tentatively determine which features seem worthwhile to explore further. In combination with studying data science techniques, I am optimistic that I can build a model that describes signifiers of successful novel discoveries about our programming.